In [ ]:
import alpaca_trade_api as tradeapi

api = tradeapi.REST('<insert api key>', '<insert secret key>', base_url='https://paper-api.alpaca.markets') 

# alpaca_trade_api Github repo: https://github.com/alpacahq/alpaca-trade-api-python/tree/cd22b3393aff8df214d867b6a4723a21ea34a3c0
# Info on bracket orders https://alpaca.markets/blog/bracket-orders/

In [ ]:
barset = api.get_barset('TSLA', 'day', limit = 10).df['TSLA']

today_close = barset['close'][-1]

# These are for long trades. Flip the numbers around for short trades.
TP = today_close*1.2 # 20% TP
SL = today_close*0.8 # 20% SL

In [ ]:
# Market order

order_info = None

order_info = api.submit_order(
    symbol='TSLA',
    side='buy',
    type='market',
    qty='1',
    time_in_force='day',
    extended_hours=False, # Has to be False
    order_class='bracket',
    stop_loss=dict(stop_price=SL), 
    take_profit=dict(limit_price=TP)
    )
        
print(order_info)

In [ ]:
# Limit order

order_info = None

order_info = api.submit_order(
    symbol='TSLA',
    side='buy',
    type='limit',
    limit_price=today_close-100,
    qty='1',
    time_in_force='day',
    extended_hours=False, # Has to be False
    order_class='bracket',
    stop_loss=dict(stop_price=SL-100), 
    take_profit=dict(limit_price=TP+100)
    )
        
print(order_info)



In [ ]:
# Additional info on how to retrieve the order ID of the TP and SL orders

if order_info.legs[0].type == 'limit':
    tp_order_id = order_info.legs[0].id
    sl_order_id = order_info.legs[1].id
    
elif order_info.legs[0].type == 'stop':
    tp_order_id = order_info.legs[1].id
    sl_order_id = order_info.legs[0].id

print(f'TP Order ID: {tp_order_id}')
print(f'TP Level: {api.get_order(tp_order_id).limit_price}')
print(f'SL Order ID: {sl_order_id}')
print(f'SL Level: {api.get_order(sl_order_id).stop_price}')

# Looking for the status change

In [ ]:
api.get_order(tp_order_id)

In [ ]:
api.get_order(sl_order_id)

In [ ]:
api.cancel_order(tp_order_id) # Cancel the TP